In [1]:
# Dependencies
from config import g_key
import gmaps
import pandas as pd
import numpy as mp
import requests
import json
import os
import gmaps.datasets

In [2]:
# Get / Read csv
data_df = pd.read_csv("../WeatherPy/Analysis.csv")
data_df.head(2)

,City,Country Code,Lat [deg],Lng [deg],Temperature [C],Humidity [%],Cloudiness [%],Wind Speed (kph]
0,albany,au,42.6001,-73.9662,17.57,91,100,0.89
1,hermanus,za,-34.4187,19.2345,19.33,47,97,1.44


In [3]:
data_df.rename(columns={"Country Code": "Country"}, inplace=True)
data_df.head(2)

,City,Country,Lat [deg],Lng [deg],Temperature [C],Humidity [%],Cloudiness [%],Wind Speed (kph]
0,albany,au,42.6001,-73.9662,17.57,91,100,0.89
1,hermanus,za,-34.4187,19.2345,19.33,47,97,1.44


In [44]:
# Configure
gmaps.configure(api_key=g_key)

# Store & Convert
loc_df = data_df[["Lat [deg]", "Lng [deg]"]]
hum_df = data_df["Humidity [%]"]





In [50]:
# Create heatmap layer

figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
gmaps.figure(layout=figure_layout)

#Heat = gmaps.heatmap_layer(loc_df, weights=hum_df, 
                                 #dissipating=False, 
                                 #point_radius = 3)

fig.add_layer(Heat)
fig

Figure(layout=FigureLayout(height='420px'))

In [51]:
# Filter weather
Good_Cities = pd.DataFrame()

for index,row in data_df.iterrows():
    if row["Temperature [C]"]>20 and row["Temperature [C]"]<35 and row['Wind Speed (kph]']<3 and row["Cloudiness [%]"]==0:
        Good_Cities[index]=row
Good_Cities.dropna().T


,City,Country,Lat [deg],Lng [deg],Temperature [C],Humidity [%],Cloudiness [%],Wind Speed (kph]
7,port elizabeth,za,-33.918,25.5701,22.16,26,0,1.03
84,presidencia roque saenz pena,ar,-26.7852,-60.4388,20.72,51,0,2.38
181,san patricio,mx,28.017,-97.5169,25.38,93,0,2.52
294,ponta do sol,cv,32.6667,-17.1,24.78,72,0,0.66
355,yafran,ly,32.0633,12.5286,32.46,15,0,1.68
377,ponta do sol,pt,32.6667,-17.1,24.78,72,0,0.66
438,karaton,kz,-6.3424,106.105,29.08,79,0,1.5
444,chisinau,md,47.0056,28.8575,29.24,37,0,2.57
545,skelleftea,se,64.7507,20.9528,29.44,40,0,0.45


In [32]:
# Dataframe for nearest hotels

hotels_df=Good_Cities.loc[["City","Country","Lat [deg]","Lng [deg]"]].T
hotels_df["Hotel Name"]=""
hotels_df["Hotel Lat"]=""
hotels_df["Hotel Lng"]=""
hotels_df["Hotel Rating"]=""
hotels_df["Hotel Total Reviews"]=""
hotels_df.head()

,City,Country,Lat [deg],Lng [deg],Hotel Name,Hotel Lat,Hotel Lng,Hotel Rating,Hotel Total Reviews
7,port elizabeth,za,-33.918,25.5701,,,,,
84,presidencia roque saenz pena,ar,-26.7852,-60.4388,,,,,
181,san patricio,mx,28.017,-97.5169,,,,,
294,ponta do sol,cv,32.6667,-17.1,,,,,
355,yafran,ly,32.0633,12.5286,,,,,


In [64]:
url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json?center=0,0&zoom=1&size=400x300"
for index,row in hotels_df.iterrows():
    
    params = {
    "location": " {} , {} ".format(row["Lat [deg]"],row["Lng [deg]"]) ,
    "rankby": "distance",
    "type": "lodging",
    "keyword": "hotel",
    "key": g_key}
    response = requests.get(url, params=params).json()
    
    # Add to DF
    hotels_df.loc[index,"Hotel Lat"]=response["results"][0]['geometry']['location']["lat"]
    hotels_df.loc[index,"Hotel Lng"]=response["results"][0]['geometry']['location']['lng']
    hotels_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
    hotels_df.loc[index,"Hotel Rating"]=response["results"][0]["rating"]
    hotels_df.loc[index,"Hotel Total Reviews"]=response["results"][0]["user_ratings_total"]

hotels_df.head(2)

,City,Country,Lat [deg],Lng [deg],Hotel Name,Hotel Lat,Hotel Lng,Hotel Rating,Hotel Total Reviews
7,port elizabeth,za,-33.918,25.5701,Newton Sands Guesthouse,-33.945,25.5648,4.7,29
84,presidencia roque saenz pena,ar,-26.7852,-60.4388,"Hotel il colono,,,",-26.7857,-60.4395,3.6,14


In [65]:
# Add to heatmap
hotels=[]
for index,row in hotels_df.iterrows():
    hotels.append( tuple([row["Hotel Lat"],row["Hotel Lng"]] )) 

markers=gmaps.marker_layer(hotels)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
loc_df = hotels_df[["Lat [deg]", "Lng [deg]"]]


In [68]:
markers1 = gmaps.marker_layer(locations=loc_df, info_box_content=hotel_info)
fig.add_layer(markers1)
fig


Figure(layout=FigureLayout(height='420px'))